**Note**: needs to be run separetely in the rtv environment (not the MBM). Computes the sky view factor from DEMs (needed as inputs in the MBM).

In [ ]:
# --- Standard library ---
import os
import glob
import shutil
import warnings

# --- Scientific / numeric ---
import numpy as np
import xarray as xr

# --- Geospatial ---
import rasterio
import rioxarray
from rasterio.transform import Affine
import rvt.default
import rvt.vis

# --- Plotting / progress ---
import matplotlib.pyplot as plt
from tqdm import tqdm

from scripts.svf_processing import *

# --- Misc settings ---
warnings.filterwarnings("ignore")
cfg_datapath = '/scratch-3/vmarijn/MassBalanceMachine/../data/'

In [ ]:
# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVF_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315

SRC_CRS = "EPSG:2056"  # LV95 (meters)
DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]

## RGI:

In [ ]:
path_geotiff = os.path.join(cfg_datapath,
                            'GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/')

rhone_rgi = "RGI60-11.01238"
rhone_path = os.path.join(path_geotiff, f"{rhone_rgi}.tif")

# Open DEM
rhone = rioxarray.open_rasterio(rhone_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
rhone.plot(cmap="terrain")
plt.title(f"DEM of Glacier {rhone_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

### Compute skyview factor:

In [ ]:
# -------------------
# Paths & parameters
# -------------------
# For this, DEMs need to be in geotiff format in LV95 (EPSG:2056)
path_geotiff = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                            "geotiff_meters_lv95/")
path_out = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11", "svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)


# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "RGI60-11.*.tif")))
    # tif_list = [
    #     '/scratch-3/vmarijn/MassBalanceMachine/../data/GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/RGI60-11.01238.tif'
    # ]
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path,
                                  out_nc,
                                  svf_n_dir=SVF_N_DIR,
                                  svf_r_max=SVF_R_MAX,
                                  svf_noise=SVF_NOISE,
                                  asvf_level=ASVF_LEVEL,
                                  asvf_dir=ASVF_DIR)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# --- Path to one of your generated SVF NetCDFs ---
path_svf = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11/svf_nc",
                        f"{rhone_rgi}_svf.nc")

# --- Load dataset ---
ds = xr.open_dataset(path_svf)

# get x and y bounds of the glacier
x_min, x_max = float(ds.x.min()), float(ds.x.max())
y_min, y_max = float(ds.y.min()), float(ds.y.max())

print(f"x: {x_min} to {x_max}")
print(f"y: {y_min} to {y_max}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("x (m)")
else:
    axes[1].set_visible(False)

# Positive Openness
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("x (m)")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{os.path.basename(path_svf)}", fontsize=14)
plt.show()

### Reproject to Lat/Lon:

In [ ]:
# -------- paths --------
path_nc_lv95 = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                            "svf_nc/")  # input: LV95 x/y .nc
path_nc_ll = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)


def main():
    files = sorted(glob.glob(os.path.join(path_nc_lv95, "RGI60-11.*_svf.nc")))
    print(f"Found {len(files)} LV95 SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(f,
                                     out_path,
                                     data_vars=DATA_VARS,
                                     src_crs=SRC_CRS,
                                     dst_crs=DST_CRS)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# Path to one of your reprojected SVF files
path_svflatlon = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                              "svf_nc_latlon", f"{rhone_rgi}_svf_latlon.nc"
                              #"RGI60-11.00001_svf_latlon.zarr"
                              )

# --- Load dataset ---
ds = xr.open_dataset(path_svflatlon)
print(ds)  # see variables, coords, attrs

# --- Extract variables (some glaciers may miss one or two) ---
svf = ds.get("svf")
asvf = ds.get("asvf")
opns = ds.get("opns")

# Compare to OGGM DEM
path_xr_grids = os.path.join(cfg_datapath,
                             "GLAMOS/topo/RGI_v6_11/xr_masked_grids/")
ds_oggm = xr.open_zarr(path_xr_grids + rhone_rgi + '.zarr')

# --- Plot ---
fig, axes = plt.subplots(1, 4, figsize=(15, 5), constrained_layout=True)

# SVF
if svf is not None:
    svf.plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
    axes[0].set_title("Sky-View Factor (SVF)")
else:
    axes[0].set_visible(False)

# ASVF
if asvf is not None:
    asvf.plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
else:
    axes[1].set_visible(False)

# Positive Openness
if opns is not None:
    opns.plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
else:
    axes[2].set_visible(False)

ds_oggm.masked_elev.plot(ax=axes[3])
axes[3].set_title("OGGM DEM")

# Common labels
for ax in axes:
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

plt.suptitle(os.path.basename(path_svflatlon), fontsize=13)
plt.show()

## SGI:
(Geotiffs are computed in SGI notebook (1.3))

In [ ]:
path_geotiff = os.path.join(cfg_datapath,
                            'GLAMOS/topo/SGI2020/DEMs_geotiff_lv95/')

aletsch_rgi = "B36-26"
aletsch_path = os.path.join(path_geotiff, f"{aletsch_rgi}.tif")

# Open DEM
aletsch = rioxarray.open_rasterio(aletsch_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
aletsch.plot(cmap="terrain")
plt.title(f"DEM of Glacier {aletsch_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

### Compute skyview factor:

In [ ]:
# -------------------
# Paths & parameters
# -------------------
path_geotiff = os.path.join(cfg_datapath,
                            "GLAMOS/topo/SGI2020/DEMs_geotiff_lv95")
path_out = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020/svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)

# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVFR_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315


# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "*.tif")))
    # tif_list = [
    #     '/scratch-3/vmarijn/MassBalanceMachine/../data/GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/RGI60-11.01238.tif'
    # ]
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path,
                                  out_nc,
                                  svf_n_dir=SVF_N_DIR,
                                  svf_r_max=SVFR_R_MAX,
                                  svf_noise=SVF_NOISE,
                                  asvf_level=ASVF_LEVEL,
                                  asvf_dir=ASVF_DIR)
        except Exception as e:
            print(f"Failed on {base}: {e}")

if __name__ == "__main__":
    main()

### Reproject to lat/lon:

In [ ]:
# -------- paths --------
path_nc_lv95 = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020",
                            "svf_nc/")  # input: LV95 x/y .nc
path_nc_ll = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020",
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)

SRC_CRS = "EPSG:2056"  # LV95 (meters)
DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]


def main():
    files = sorted(glob.glob(os.path.join(path_nc_lv95, "*_svf.nc")))
    print(f"Found {len(files)} LV95 SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(f, out_path, data_vars=DATA_VARS,src_crs=SRC_CRS,dst_crs=DST_CRS)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

## GLAMOS DEMs:

In [ ]:
path_geotiff = os.path.join(cfg_datapath,
                            'GLAMOS/topo/GLAMOS_DEM/DEMs_geotiff_lv95/')

aletsch_rgi = "aletsch"
aletsch_path = os.path.join(path_geotiff, f"{aletsch_rgi}_2016.tif")

# Open DEM
aletsch = rioxarray.open_rasterio(aletsch_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
aletsch.plot(cmap="terrain")
plt.title(f"DEM of Glacier {aletsch_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

In [ ]:
aletsch_rgi = "gietro"
aletsch_path = os.path.join(path_geotiff, f"{aletsch_rgi}_2016.tif")

# Open DEM
aletsch = rioxarray.open_rasterio(aletsch_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
aletsch.plot(cmap="terrain")
plt.title(f"DEM of Glacier {aletsch_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

### Compute skyview factor:

In [ ]:
# -------------------
# Paths & parameters
# -------------------
path_geotiff = os.path.join(cfg_datapath,
                            "GLAMOS/topo/GLAMOS_DEM/DEMs_geotiff_lv95")
path_out = os.path.join(cfg_datapath, "GLAMOS/topo/GLAMOS_DEM/svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)

# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVFR_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315

# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "*.tif")))
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path,
                                  out_nc,
                                  svf_n_dir=SVF_N_DIR,
                                  svf_r_max=SVFR_R_MAX,
                                  svf_noise=SVF_NOISE,
                                  asvf_level=ASVF_LEVEL,
                                  asvf_dir=ASVF_DIR)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

### Reproject to lat/lon:

In [ ]:
# -------- paths --------
path_nc_lv95 = os.path.join(cfg_datapath, "GLAMOS/topo/GLAMOS_DEM",
                            "svf_nc/")  # input: LV95 x/y .nc
path_nc_ll = os.path.join(cfg_datapath, "GLAMOS/topo/GLAMOS_DEM",
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)

SRC_CRS = "EPSG:2056"  # LV95 (meters)
DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]


def main():
    files = sorted(glob.glob(os.path.join(path_nc_lv95, "*_svf.nc")))
    print(f"Found {len(files)} LV95 SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(f, out_path, data_vars=DATA_VARS,src_crs=SRC_CRS,dst_crs=DST_CRS)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()